# Обработка текстов

Вам дан файл `ru.conversations.txt`

В нем содержатся диалоги (разделены пустой строкой).

Вам необходимо:

### Разбить текст на диалоги

In [20]:
with open('ru.conversations.txt', 'r', encoding='utf-8') as f:
    text = []
    for line in f:
        text.append(line)

In [21]:
text[:10]

['- Что,  Мансур,  не жарко теперь тебе?\n',
 '- Спрашиваю, не жарко ему теперь?\n',
 '\n',
 '- Какой полк?\n',
 '- Тысяча тридцать четвертый.\n',
 '- Вези дальше. Тут тысяча двадцать шестой.\n',
 '\n',
 '- Какая санрота?\n',
 '- Тысяча тридцать шестая.\n',
 '- Значит, наша! Принимай тяжелораненого!\n']

In [22]:
dialogs = []
dialog = ''
for item in text:
    if item != '\n':
        dialog += item
    else:
        dialogs.append(dialog)
        dialog = ''
print(dialogs[:5])

['- Что,  Мансур,  не жарко теперь тебе?\n- Спрашиваю, не жарко ему теперь?\n', '- Какой полк?\n- Тысяча тридцать четвертый.\n- Вези дальше. Тут тысяча двадцать шестой.\n', '- Какая санрота?\n- Тысяча тридцать шестая.\n- Значит, наша! Принимай тяжелораненого!\n', '- Сколько фрицев в котле?\n- Тысяч сорок.\n', '- Чем же он отравился?\n- А вон, видишь, что-то из тех бутылей выпил.\n']


### Разбить диалоги на токены

In [23]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
for i,item in enumerate(dialogs):
    dialogs[i] = word_tokenize(item)

### Предобработать токены
Удалить стоп-слова и спецсимволы, лемматизировать или стеммировать слова и т.д.

In [26]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
tokenizer = RegexpTokenizer(r'\w+')
for i,item in enumerate(dialogs):
    dialogs[i] = tokenizer.tokenize(str(dialogs[i]))

In [29]:
clean_dialogs = dialogs
for i, dialog in enumerate(dialogs):
    clean_dialog = dialog
    for word in dialog:
        if word.lower() in stopwords.words('russian'):
            clean_dialog.remove(word)
    clean_dialogs[i] = clean_dialog

In [30]:
clean_dialogs[:5]

[['Мансур', 'жарко', 'тебе', 'Спрашиваю', 'жарко', 'теперь'],
 ['полк',
  'Тысяча',
  'тридцать',
  'четвертый',
  'Вези',
  'дальше',
  'тысяча',
  'двадцать',
  'шестой'],
 ['санрота',
  'Тысяча',
  'тридцать',
  'шестая',
  'Значит',
  'наша',
  'Принимай',
  'тяжелораненого'],
 ['Сколько', 'фрицев', 'котле', 'Тысяч', 'сорок'],
 ['же', 'отравился', 'вон', 'видишь', 'то', 'тех', 'бутылей', 'выпил']]

In [31]:
from nltk.stem import SnowballStemmer
rus_stemmer = SnowballStemmer('russian')

In [33]:
stem_dialogs = clean_dialogs
for i, dialog in enumerate(clean_dialogs):
    stem_dialog = dialog
    for j, word in enumerate(dialog):
        stem_dialog[j] = rus_stemmer.stem(word)
    stem_dialogs[i] = stem_dialog

### Перевести каждый из диалогов в векторное представление

С помощью TF-IDF представить каждый из диалогов в векторном формате

In [34]:
import sklearn.feature_extraction.text as txt

In [36]:
strings = [' '.join(map(str, stem_dialogs[i])) for i in range(len(stem_dialogs))]

In [37]:
strings[:10]

['мансур жарк теб спрашива жарк тепер',
 'полк тысяч тридца четверт вез дальш тысяч двадца шест',
 'санрот тысяч тридца шест знач наш принима тяжелоранен',
 'скольк фриц котл тысяч сорок',
 'же отрав вон вид то тех бутыл вып',
 'что мансур дел фашист прорва оборон мы тоб спим',
 'их гумрак',
 'комсорг не прот ещ сто',
 'брал во',
 'мансур ты сбил может нибуд стреля']

In [39]:
tfvec = txt.TfidfVectorizer()
vect = tfvec.fit_transform(strings[:1000]).toarray()

In [40]:
vect

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Найти самые близкие по содержанию диалоги, исходя из векторного представления

In [41]:
from scipy.spatial.distance import euclidean

In [42]:
euclid = []
for i in range(1000):
    for j in range(i):
        euclid.append((i,j, euclidean(vect[i], vect[j])))

In [43]:
top = sorted(euclid, key=lambda x: x[2])[:10]
top

[(23, 22, 0.0),
 (660, 659, 0.0),
 (732, 113, 0.0),
 (548, 547, 0.39714431308522974),
 (571, 434, 0.5076919943988746),
 (625, 603, 0.5399726721243513),
 (293, 156, 0.5740216055422772),
 (663, 69, 0.6632243265416782),
 (202, 159, 0.7027577435761159),
 (943, 526, 0.7150294095118789)]

In [44]:
index0 = [item[0] for item in top]
index1 = [item[1] for item in top]

In [45]:
for i,j in zip(index0,index1):
        print(strings[i], '~', strings[j])

плох наш дел над идт ~ плох наш дел над идт
кажет ошиб то провер да ~ кажет ошиб то провер да
что нич ~ что нич
знает вы майор шелл знает вы майор онезорг ~ знает вы майор шелл знает вы капита онезорг
кто говор говор ~ говор говор говор
неужел эт правд правд ~ правд эт правд что эт
нашл ~ как нашл клад нашл
ясн товарищ капита товарищ капита товарищ капита как будут указан никак по самолет ~ я товарищ капита
разреш нача начина ~ разреш начина начина
понятн понятн вам понятн фашист бок правильн ~ понятн глущенк понятн


In [48]:
dialogs = []
dialog = ''
for item in text:
    if item != '\n':
        dialog += item
    else:
        dialogs.append(dialog)
        dialog = ''

In [50]:
for i,j in zip(index0,index1):
        print(dialogs[i], '~~~~~~~\n', dialogs[j])

- Плохо наше дело.
- Опять надо идти?
 ~~~~~~~
 - Плохо наше дело.
- Опять надо идти?

- Я кажется ошибся где-то.
- А проверить можно?
- Да,  конечно.
 ~~~~~~~
 - Я кажется ошибся где-то.
- А проверить можно?
- Да,  конечно.

- Ну, что там?
- Ничего.
 ~~~~~~~
 - Есть что-нибудь?
- Ничего.

- Знаете ли вы майора Шелла?
- Нет.
- Знаете ли вы майора Онезорге?
- Нет.
 ~~~~~~~
 - Знаете ли вы майора Шелла?
- Нет.
- Знаете ли вы капитана Онезорге?
- Нет.

- А кто говорит?
- Все говорят!
 ~~~~~~~
 - Ну,  говори же, говори!
- Говорить надо?

- Да неужели это правда?
- Да, правда.
 ~~~~~~~
 - Правда ли это?
- Правда.
- Ну  что  же,  это  хорошо.

- Нашли?
- Нет!..
 ~~~~~~~
 - Ну как, нашли клад?
- Нашли.

- Ясно, товарищ капитан.
- Товарищ  капитан.
- Есть, товарищ капитан.
- Какие будут еще указания?
- Больше никаких.
- Тогда  по  самолетам.
 ~~~~~~~
 - Кто?
- Я,товарищ капитан!

- Разрешите начать?
- Начинайте!
 ~~~~~~~
 - Разрешите начинать?
- Начинайте!

- Понятно?
- Понятно.
- Что вам поня